In [1]:
import database_connected_naver as dcn
import time, random
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

def get_ext2(detail_price, ext1):
    if 'kg' in ext1:
        hund_price = float(detail_price.replace(',', ''))
        weight = float(ext1.replace('kg', ''))
        weight = weight * 1000
        return str(int(hund_price / weight * 1000))
    elif 'g' in ext1:
        hund_price = float(detail_price.replace(',', ''))
        weight = float(ext1.replace('g', ''))
        return str(int(hund_price / weight * 1000))
    return ''
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)

In [ ]:
page = 2100
pagelimit = 700
temp_list = dcn.select_data({'page': page, 'pagelimit': pagelimit})
for temp in temp_list:
    parent_pcode = temp[0]
    driver.get(f'https://search.shopping.naver.com/catalog/{parent_pcode}')
    time.sleep(random.randint(1, 2))
    try:
         style_detail = driver.find_element_by_css_selector('div[class^="style_detail"]')
    except NoSuchElementException:
        continue
    ext_list = style_detail.find_elements_by_css_selector('div[class^="filter_condition_group"] > ul > li')

    for i, ext in enumerate(ext_list):
        detail_price = ext.find_element_by_css_selector('span[class^="filter_price"]').text
        detail_price = detail_price.replace('모바일', '')
        detail_price = detail_price.replace('원', '')
        detail_price = detail_price.replace(',','')
        ext1 = ext.find_element_by_css_selector('span[class^="filter_text"]').text
        ext2 = get_ext2(detail_price, ext1)
        pcode = str(parent_pcode)+'-'+ext1
        dcn.push_prod_data_detail({
            'parent_pcode': parent_pcode,
            'pcode': pcode,
            'price': detail_price,
            'weight': ext1,
            'price_per_kg': ext2,
        })
    time.sleep(random.randint(1, 3))

In [3]:
driver.quit()